# 푸아송 분포를 통한 언급 급상승 키워드 추출

In [5]:
import pandas as pd
from konlpy.tag import Mecab
from collections import Counter
from scipy.stats import poisson

In [6]:
df = pd.read_csv('../extract/dcinside/result/dcinside_avante.csv')
# df = pd.read_csv('../extract/dcinside/result/merged_naver_cafe_data.csv')
# df = pd.read_csv('../extract/dcinside/result/bobaedream_아이오닉_Extract_Result.csv')
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Title'] = df['Title'].fillna('')
df['Body'] = df['Body'].fillna('')
df['Comment'] = df['Comment'].fillna('')
df.head()

,Date,Time,Title,Body,Comment,View,Like,Community,CarName
0,2019-01-01,00:59,아반떼 페리 되고 나서,\nA8이 이뻐 보임...,,146.0,0.0,dcinside,아반떼
1,2019-01-01,03:38,시발 취업되면 아반떼라도 사고 싶다,중고 모닝쟝을 1년 더 타야 하는대,타던 모닝 나에게 팔지 않을래?\n아반떼 hd 200만원 ㄱㄱ 못생긴거 빼면 단점이...,141.0,0.0,dcinside,아반떼
2,2019-01-01,09:00,새해에는 모닝에서 벗어나고싶다,아반떼 타고시퍼,조촐하네\n모닝타다가 아반떼타면 신세계지,105.0,0.0,dcinside,아반떼
3,2019-01-01,10:44,본인 182에 105키로 눈마주치면 바로눈깜,병신찐따새끼들 걍약약강 존나많음 ㅋㅋ\n차도 좆병신같은거 타는새끼들이 특히많음\n모...,역시 짱용차를 타는사람은 다르군여\n닉갑ㄴ,116.0,0.0,dcinside,아반떼
4,2019-01-01,11:24,차고민좀 들어주세요,좆닝타는데\n이번에 돈좀생겨서 아반떼로 갈랫더니\n스마트 풀옵이 케파랑 돈 400차...,지랄말고 케파 가자!!\n할부할거면 걍 중형타던가 아니면 아방스 가시던지\n다 일시...,161.0,0.0,dcinside,아반떼


In [7]:
# https://docs.google.com/spreadsheets/d/1-9blXKjtjeKZqsf4NzHeYJCrr49-nXeRF6D80udfcwY/edit?gid=589544265#gid=589544265
mecab = Mecab()

def get_nouns(text):
    if not isinstance(text, str):
        return []
    
    tokens = mecab.pos(text)
    nouns = [word for word, pos in tokens if pos.startswith('NNG') or pos.startswith('NNP') or pos.startswith('SL')]
    return list(set(nouns))

# 게시글 제목과 내용을 합친 후 명사 추출
TITLE_WEIGHT = 5
BODY_WEIGHT = 2
COMMENT_WEIGHT = 1
df['nouns_title'] = df['Title'].apply(get_nouns)
df['nouns_body'] = df['Body'].apply(get_nouns)
df['nouns_comment'] = df['Comment'].apply(get_nouns)
df['nouns'] = TITLE_WEIGHT * df['nouns_title'] + BODY_WEIGHT * df['nouns_body'] + COMMENT_WEIGHT * df['nouns_comment']
df[['Date', 'Title', 'Body', 'nouns']].head()

,Date,Title,Body,nouns
0,2019-01-01,아반떼 페리 되고 나서,\nA8이 이뻐 보임...,"[아반떼, 페리, 아반떼, 페리, 아반떼, 페리, 아반떼, 페리, 아반떼, 페리, ..."
1,2019-01-01,시발 취업되면 아반떼라도 사고 싶다,중고 모닝쟝을 1년 더 타야 하는대,"[시발, 취업, 아반떼, 시발, 취업, 아반떼, 시발, 취업, 아반떼, 시발, 취업..."
2,2019-01-01,새해에는 모닝에서 벗어나고싶다,아반떼 타고시퍼,"[새해, 모닝, 새해, 모닝, 새해, 모닝, 새해, 모닝, 새해, 모닝, 아반떼, ..."
3,2019-01-01,본인 182에 105키로 눈마주치면 바로눈깜,병신찐따새끼들 걍약약강 존나많음 ㅋㅋ\n차도 좆병신같은거 타는새끼들이 특히많음\n모...,"[키, 깜, 눈, 본인, 키, 깜, 눈, 본인, 키, 깜, 눈, 본인, 키, 깜, ..."
4,2019-01-01,차고민좀 들어주세요,좆닝타는데\n이번에 돈좀생겨서 아반떼로 갈랫더니\n스마트 풀옵이 케파랑 돈 400차...,"[고민, 차, 고민, 차, 고민, 차, 고민, 차, 고민, 차, 스마트, 고민, 랫..."


In [8]:
# # 날짜별 단어 등장 및 게시글 개수 계산
grouped = df.groupby('Date').agg(nouns=('nouns', 'sum'), post_count=('Date', 'size'))
word_count_by_date = grouped['nouns'].apply(lambda x: dict(Counter(x)))
wc_df = pd.DataFrame({'Date': grouped.index, 'Word_Count': word_count_by_date, 'Post_Count': grouped['post_count']})
wc_df.head()

,Date,Word_Count,Post_Count
Date,,,
2019-01-01,2019-01-01,"{'아반떼': 44, '페리': 5, 'A': 2, '시발': 5, '취업': 5,...",11
2019-01-02,2019-01-02,"{'학식': 8, '감각': 6, '현실': 5, '시절': 6, '소나타': 10...",17
2019-01-03,2019-01-03,"{'학식': 5, '민주': 5, '유지비': 12, '차': 26, 'hd': 2...",17
2019-01-04,2019-01-04,"{'편': 5, '차': 66, '돌이': 6, '엑센트': 7, '국장': 2, ...",26
2019-01-05,2019-01-05,"{'아반떼': 76, 'XD': 5, '패드': 2, '골드': 2, '때': 7,...",19


In [9]:
SHORT_TERM = 7
LONG_TERM = 365
TODAY = pd.to_datetime('2024-3-20')

short_term_start_date = TODAY - pd.to_timedelta(SHORT_TERM, unit='D')
long_term_start_date = TODAY - pd.to_timedelta(LONG_TERM, unit='D')

short_term_df = wc_df[(short_term_start_date < wc_df['Date']) & (wc_df['Date'] <= TODAY)]
long_term_df = wc_df[(long_term_start_date < wc_df['Date']) & (wc_df['Date'] <= TODAY)]

# 모든 단어의 개수 합산
def aggregate_word_counts(df):
    total_count = Counter()
    for word_count in df['Word_Count']:
        total_count.update(word_count)
    return dict(total_count)

short_term_word_count = aggregate_word_counts(short_term_df)
long_term_word_count = aggregate_word_counts(long_term_df)
short_term_post_count = short_term_df['Post_Count'].sum()
long_term_post_count = long_term_df['Post_Count'].sum()
short_term_post_count, long_term_post_count

(np.int64(275), np.int64(15892))

In [16]:
filter_words = {
    'ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ','vs','BH','G','g','e','','','',

    '현대자동차', '기아', '현차', '현대차', '모델', '전기차', '자동차', '중고', '차', '현기', '내연', '운전', '현대', '차종', '신차', '국산', '르노', '렉서스', '', 
    '아이오닉', '제네시스', '아반떼', '니로', '벤츠', '토레스', '쏘나타', '카니발', '하이브리드', '모닝', '봉고', '테슬라', '포르쉐', '레이', '쏘렌토', '', '', 

    '차이', '돈', '밥', '이유', '글', '집', '세대', '고민', '내년', '올해', '비교', '개', '값', '닉', '급', '생각', '뒤', '기준', '현재', '사람', '애', '카',
    '회사', '시리즈', '속도', '키', '코', '가능', '시작', '오늘', '사진', '봄', '차량', '정도', '기본', '탐', '일반', '브랜드', '막', '나라', '택', '풀',
    '앞', '뒤', '추천', '말', '맘',  '땅', '주변', '추가', '신', '최고', '기사', '길', '순서', '도로', '초반', '기차', '미래', '배', '세계', '일', '자체',
    '테', '독', '상실', '만약', '필요', '베', '느낌', '렌', '랜', '지금', '이번', '노인', '페리', '형', '다시', '렁', '듬', '져', '인', '전', '쌈', '때', '트', '감상', 
    '따리', '뉴', '점', '특유', '동안', '엔', '칸', '특징', '인생', '정보', '예정', '롱', '주', '전자', '다음', '시속', '르', '뉘', '쉐', '츄', '쌍', '줄', '혼', '', '', '', '', 
    '좆', '병신', '시발', '지랄', '새끼', '색', '결혼', '타협', '후기', 'lf', '투표', '도전', '후반', '비용', '종류', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 
}

MIN_COUNT = TITLE_WEIGHT + BODY_WEIGHT + COMMENT_WEIGHT + 1

filtered_short_term_word_count = dict((word, count) for word, count in short_term_word_count.items() if word not in filter_words and count >= MIN_COUNT)
filtered_long_term_word_count = dict((word, count) for word, count in long_term_word_count.items() if word not in filter_words and count >= MIN_COUNT)

In [17]:
# 각 단어의 등장 횟수를 푸아송 분포로 계산
def calculate_poisson_scores(short_term_word_count, long_term_word_count, short_term_post_count, long_term_post_count):
    poisson_scores = []
    for word, short_count in short_term_word_count.items():
        # short_count /= short_term_post_count
        # long_count = long_term_word_count.get(word, 0) / long_term_post_count
        long_count = long_term_word_count.get(word, 0)

        expected_short_count = long_count * SHORT_TERM / LONG_TERM
        lambda_ = long_count / len(long_term_df)
        score = poisson.pmf(short_count, expected_short_count)
        poisson_scores.append((word, short_count, int(expected_short_count), long_count, score))
    return poisson_scores

# 푸아송 점수를 계산하고 데이터프레임으로 변환
poisson_scores = calculate_poisson_scores(filtered_short_term_word_count, filtered_long_term_word_count, short_term_post_count, long_term_post_count)
poisson_df = pd.DataFrame(poisson_scores, columns=['Word', 'Short_Count', 'Expteted_Short_Count', 'Long_Count', 'Poisson_Score'])
poisson_df = poisson_df.sort_values(by='Poisson_Score', ascending=True)

poisson_df.head(30)

,Word,Short_Count,Expteted_Short_Count,Long_Count,Poisson_Score
16,방증,56,1,58,1.795190e-73
168,예열,11,0,13,4.516499e-15
75,코인,18,1,73,1.644188e-14
29,프라이드,15,1,93,7.554944e-10
58,캐스퍼,52,19,1030,7.725971e-10
0,eq,13,1,81,1.041954e-08
93,트랙스,24,61,3219,2.339265e-08
8,하브,15,45,2371,1.003797e-07
181,대별,10,1,61,4.106975e-07
167,문,16,3,180,6.157649e-07


In [87]:
# 상위 5개 단어 추출
top_words = poisson_df['Word'][:5]

# short_term_df 생성
link_df = df[(short_term_start_date < df['Date']) & (df['Date'] <= TODAY)]

# 단어가 포함된 게시글의 링크 추출 및 출력
for word in top_words:
    filtered_links = link_df[link_df[['Title', 'Body']].apply(lambda x: word in x['Title'] or word in x['Body'], axis=1)]['Url']
    print(f"{word}: {filtered_links.tolist()[:3]}")

도장: ['https://gall.dcinside.com/board/view/?id=car_new1&no=9151732', 'https://gall.dcinside.com/board/view/?id=car_new1&no=9167659', 'https://gall.dcinside.com/board/view/?id=car_new1&no=9167896']
돌: ['https://gall.dcinside.com/board/view/?id=car_new1&no=9151918', 'https://gall.dcinside.com/board/view/?id=car_new1&no=9154917', 'https://gall.dcinside.com/board/view/?id=car_new1&no=9163474']
고속도: ['https://gall.dcinside.com/board/view/?id=car_new1&no=9156191', 'https://gall.dcinside.com/board/view/?id=car_new1&no=9156494', 'https://gall.dcinside.com/board/view/?id=car_new1&no=9160664']
디스플레이: ['https://gall.dcinside.com/board/view/?id=car_new1&no=9164294', 'https://gall.dcinside.com/board/view/?id=car_new1&no=9166373', 'https://gall.dcinside.com/board/view/?id=car_new1&no=9171397']
코인: ['https://gall.dcinside.com/board/view/?id=car_new1&no=9152163']
